Karpathy has CHALLENGED US TO BEAT HIS VALIDATION LOSS OF 2.2 (dev set loss).
WE MUST!!! ONWARDS!!!! (We'll change the hyperparameters hbehehe)
Also, he said we should be able to read a good chunk of the paper by now, so let's try that too :D

Anyways, for this, let's make nice functions where we can change all the hyperparameters.
Batch size, # neurons, C embedding size, lr, ... 


In [6]:
import torch
import torch.nn.functional as F
import matplotlib.pyplot as plt
%matplotlib inline

In [7]:
# Read words and generate our char <-> int functions
words = open('../2 - makemore/names.txt', 'r').read().splitlines() # Read everything
chars = sorted(list(set(''.join(words))))
stoi = {s:i+1 for i, s in enumerate(chars)} # string to integer
stoi['.'] = 0
itos = {i: s for s, i in stoi.items()} # int to string

words[:8] # woo


['emma', 'olivia', 'ava', 'isabella', 'sophia', 'charlotte', 'mia', 'amelia']

Making our dataset splits

In [8]:
def generate_dataset(words, block_size, doprint=False):
	"""returns X, Y datasets based on words list
	block_size: context length: how many characters do we take to predict the next one?
	"""
	X, Y = [], []

	for w in words:
		if doprint:
			print(w)	
		context = [0] * block_size # this would make [0, 0, ...] based on block_size

		for ch in w + '.': 
			ix = stoi[ch] # index of char
			X.append(context)
			Y.append(ix)
			
			if doprint:
				print(''.join(itos[i] for i in context), '------>', itos[ix])
			context = context[1:] + [ix] # [0,0,0] -> [0,0, ix] like a rolling effect

	X = torch.tensor(X)
	Y = torch.tensor(Y)
	print(X.shape, Y.shape)
	return X, Y

In [222]:
### Hyperparameters!!! CHANGE STUFF HERE
block_size = 3 # num of past chars to use to predict next char
embedding_size = 10
num_neurons = 200

In [223]:
# Generate our train, dev, and test loss. We'll be trying to get dev loss < 2.2
import random
random.seed(42)
random.shuffle(words)
n1 = int(0.8*len(words))
n2 = int(0.9*len(words))
print(f'number of words: {len(words)}')

Xtr, Ytr = generate_dataset(words[:n1], block_size)
Xdev, Ydev = generate_dataset(words[n1:n2], block_size)
Xte, Yte = generate_dataset(words[n2:], block_size)
# Of course, the sizes are way more massive than # word examples because these are the examples from the words (char stuff), not the words themselves.

number of words: 32033
torch.Size([182400, 3]) torch.Size([182400])
torch.Size([22962, 3]) torch.Size([22962])
torch.Size([22784, 3]) torch.Size([22784])


Next up, initialisation of our model! I.e, we'll initialise the weights.

In [224]:
g = torch.Generator().manual_seed(2147483647) # This will be a 'global' generator
def generate_parameters(block_size, embedding_size, num_neurons):
	# We booosting our hidden layer size
	C = torch.randn(27, embedding_size, generator=g) # Generate embedding of characters, e.g (27,10)

	W1 = torch.randn((embedding_size * block_size, num_neurons), generator=g) # first weights takes input from 
	b1 = torch.randn(num_neurons, generator=g)
	W2 = torch.randn((num_neurons, 27), generator=g)
	b2 = torch.randn(27, generator=g)
	parameters = [C, W1, b1, W2, b2] # for easy summing parameters

	for p in parameters:  # Turn on requires grad for our parameter matrices
		p.requires_grad = True
		
	print("Number of parameters in total:", sum(p.nelement() for p in parameters)) # num of parameters in total

	return parameters
parameters = generate_parameters(block_size, embedding_size, num_neurons)
C, W1, b1, W2, b2 = parameters
C.shape, W1.shape


Number of parameters in total: 11897


(torch.Size([27, 10]), torch.Size([30, 200]))

Just as a consolidation bit, we'd be wanting to do this to push it through the first layer
```
C[Xtr] @ W1 + b1 # -> doesn't work, as C[Xtr].shape -> torch.Size([182484, 3, 10]) || Each example has 3 inputs, and each char input has an embedding size of 10.
W1 -> torch.Size([30, 200]). 
```
We want to flatten the embedded stuff so each example just reads as a 30 length vector (or w.e the size of the thingy is), i.e ([# examples, block*embedding sizes ]). We do this via: 
```
C[Xtr].view((-1, block_size*embedding_size)).shape -> torch.Size([182484, 30])
C[Xtr].view((-1, block_size*embedding_size)) @ W1 + b1 # works! -> torch.Size([182484, 200]). Yay!
```

## And now we'll have to do gradient descent on the model. 

(this is called the training loop or whatever)
If I remember well,

0th step; we can minibatch! To make the mfer run faster, even at some loss of accuracy of the gradient to descend, it's still much better overall.

#### First step, is forward passing.
 We get the training data (specifically, training data from minibatch). We embed it, and flatten that embedding (using the efficient .view(..)), and then 'run it through' the layers by doing our activation on it. Then we run it through the second layer (our final one), giving us the logits.

We can get the loss straight up by doing F.cross_entropy, or we can get the probability dist. by softmaxing it. (And then you could work out loss. Remember, these functions are doing very simple things we've already done!! -- they just exp, normalise weights, and then get the negative loss likelihood).

#### The backwards pass
Make sure to set all our grads to 0.
Then we just  loss.backward(), propagating through every operation done and getting it's gradient w.r.t loss

#### Updating weights
And we just update the weights now :D 


In [226]:
# Implemented hardcode first

def train(X: torch.Tensor, Y: torch.Tensor, num_steps: int, block_size: int, embedding_size: int, parameters): 
	"""_summary_

	Args:
		X (torch.Tensor): X dataset to train on (train with our Xtr!)
		Y (torch.Tensor): Y dataset for labels
		num_steps (int): Number of times to go through loop
		block_size (int): Context length; # characters used to predict next one
		embedding_size (int): Size of embedding for the characters
		parameters (_type_): param


	"""
	C, W1, b1, W2, b2 = parameters

	for i in range(num_steps):
		ix = torch.randint(0, X.shape[0], (32,)) # We get {minibatch_size} random ints from size of 0- training set
		
		# forward pass -- on our minibatch :o !!!!
		emb = C[X[ix]] # (32,3,2) here || (minibatch_size, block_size, embed size))
		# print(f'{emb.shape=}')
		h = torch.tanh(emb.view(-1, block_size*embedding_size) @ W1 + b1) # We do the whole embed flatten, push it through layer 1, and tanh activation on it. All elements of h is between [-1, 1]
		logits = h @ W2 + b2 # (32, 27)


		loss = F.cross_entropy(logits, Y[ix]) # Does the epic normalisation stuff for us 
		# print(f'Loss during this minibatch loop: {loss=}')

		# backward pass - zero grad, backprop
		for p in parameters:
			p.grad = None
		loss.backward()

		# update
		# lr = lrs[i]
		lr = 0.1 if i < 20000 else 0.01
		for p in parameters:
			p.data += -lr * p.grad

		# track stats (optional part)

	return loss # Not needed, I'm just putting it here because why not

In [227]:
train_loss = train(Xtr, Ytr, 30000, block_size, embedding_size, parameters) # 
print(train_loss)

tensor(2.2736, grad_fn=<NllLossBackward0>)


### Great! Now let's evaluate our model!
(Our model is our parameters :ooooo)


In [ ]:
def evaluate_loss(X, Y, block_size, embedding_size, parameters):
	C, W1, b1, W2, b2 = parameters
	emb = C[X] # (X.shape[0],3,2) here -- whole set in this case
	h = torch.tanh(emb.view(-1, block_size*embedding_size) @ W1 + b1) # (blah, 100)

	logits = h @ W2 + b2 # (32, 27)

	loss = F.cross_entropy(logits, Y) # Does the epic normalisation stuff
	print(f'{loss=}')
	return loss